## Convolutional Neural Networks (CNN)

## TBD Doing better on Fashion MNIST dataset using CNN

In [ ]:
from keras.datasets import fashion_mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import SGD

(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()

# Normalized features
X_train = X_train.reshape(60000,28,28,1)/255.0
X_test = X_test.reshape(10000,28,28,1)/255.0

# Encode Y as binary class vector
Y_train = np_utils.to_categorical(Y_train, 10)
Y_test = np_utils.to_categorical(Y_test, 10)

# TBD: Use following architecture to classify fashion MNIST dataset
# activation function is relu for all layers except last where its softmax
# 1 Convolutional Layer (filters = 64, kernel_size = 4 )
# 2 Convolutional Layer (filters = 64, kernel_size = 4)
# 3 Max Pooling Layer (size 2)
# 4 Convolutional Layer (filters = 128, kernel_size = 3)
# 5 Convolutional Layer (filters = 128, kernel_size = 3)
# 6 Max Pooling Layer (size 2)
# 7 Dense Layer (64 neurons)
# 8 Dense Layer (10 neurons, activation = softmax)
model = Sequential([
    Conv2D(64, 3, activation="relu", padding="same", input_shape=[28, 28, 1]),
    Conv2D(64, 4, activation="relu", padding="same"),
    MaxPooling2D(2),
    Conv2D(128, 3, activation="relu", padding="same"),
    Conv2D(128, 3, activation="relu", padding="same"),
    MaxPooling2D(2),
    Flatten(),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(10, activation="softmax")
])

model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9), metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=100, epochs=10, verbose=1, validation_split=0.2)

In [ ]:
model.evaluate(X_test, Y_test)

## Recurrent Neural Networks(RNNs) 

## Sequence Prediction

## Simple RNN


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error
from sklearn.linear_model import LinearRegression
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from keras.optimizers import SGD
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

def ts2XY(ts, n):
    '''
    Generates a features and labels from a timeseries sequence
    ignores the timestamp, just arranges the sequence
    into n features by shifting 1 for each feature
    also removes the rows which have NaN due to shifting (first n  rows)
    (no error handling for now)
    :param ts: univariate timeseries dataframe, with a 'value' column
    :param n: number of features
    :return: X , Y
    '''

    X = pd.concat(
          [
            ts['value'].shift(i).rename(f'feature_{i}')
              for i in range(1, n+1)
          ], axis = 1)

    Y = ts['value'][:].to_frame()
    Y = ts['value'][n:].to_frame()
    X = X[n:]
    return X,Y



def evalts(ts, lookback, scaler, metric, test_size=0.25):

    if scaler is not None:
        ts['value'] = scaler.fit_transform(ts['value'].to_frame())

    X,Y = ts2XY(ts, lookback)

    assert(len(X) == len(Y) and 0.0 <= test_size <= 1.0 )

    f = int(len(X)*test_size)

    X_train,X_test,Y_train,Y_test = X[:f],X[f:],Y[:f],Y[f:] #because we want to preserve the order

    Y_hat1 = X_test.iloc[:, -1]
    print(f"model1 (last observation) {metric.__name__} = {metric(Y_test, Y_hat1)}")

    Y_hat2 = X_test.mean(axis=1)
    print(f"model2 (mean of lookback observations) {metric.__name__} = {metric(Y_test, Y_hat2)}")

    model3 = LinearRegression()
    model3.fit(X_train, Y_train)
    Y_hat3 = model3.predict(X_test)
    print(f"model3 (Linear regression) {metric.__name__} = {metric(Y_test, Y_hat3)}")

    model4 = Sequential([
      SimpleRNN(1, input_shape=[None, 1])
    ])

    model4.compile(loss="mse", optimizer=SGD(lr=0.01, momentum=0.9))
    X_train_ts = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
    model4.fit(X_train_ts, Y_train, epochs=100, verbose=0)
    Y_hat4 = model4.predict(np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1))
    print(f"model4 (RNN 1) {metric.__name__} = {metric(Y_test, Y_hat4)}")

    model5 = Sequential([
        SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
        SimpleRNN(20, return_sequences=True),
        SimpleRNN(1)
        # SimpleRNN(20),
        # Dense(1, activation='relu')
    ])

    model5.compile(loss="mse", optimizer=SGD(lr=0.01, momentum=0.9))
    model5.fit(X_train_ts, Y_train, epochs=10, verbose=0)
    Y_hat5 = model5.predict(np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1))
    print(f"model5 (RNN n) {metric.__name__} = {metric(Y_test, Y_hat5)}")

ts = pd.read_csv('https://raw.githubusercontent.com/numenta/NAB/master/data/artificialNoAnomaly/art_daily_no_noise.csv')
evalts(ts, 7, MinMaxScaler(), mean_squared_error)